In [1]:
import os
from pathlib import Path

from pytube import YouTube

from yt_audio_collector.constants import BASE_PATH, BASE_URL


In [2]:
video_id = "6cFo3PEr4zY"
url = f"{BASE_URL}{video_id}"
you_tube = YouTube(url)
duration = you_tube.length
try:
    # Fetching the audio of the video with given id
    video = you_tube.streams.filter(only_audio=True).first()
except Exception as exception:
    print(exception)
destination = BASE_PATH / "temp"
# Downloading the audio in temporary file and it gives audio file
# but with mp4(video supported format) extension which we need to
# convert to mp3(audio supported format)
output_file = video.download(output_path=destination)
# Renaming the downloaded file with mp3
new_file = destination / f"{video_id}.mp3"
os.rename(output_file, new_file)

In [3]:
from __future__ import print_function
import numpy as np
import librosa
import soundfile as sf

In [4]:
from os import path
from pydub import AudioSegment

input_file = "temp/6cFo3PEr4zY.mp3"
output_file = "temp/result.wav"
try:
    sound = AudioSegment.from_file(input_file, 'mp3')
except:
    sound = AudioSegment.from_file(input_file, format='mp4')
sound.export(output_file, format="wav")


<_io.BufferedRandom name='temp/result.wav'>

In [5]:

y, sr = librosa.load('temp/result.wav'
                     )


# And compute the spectrogram magnitude and phase
S_full, phase = librosa.magphase(librosa.stft(y))

In [6]:

S_filter = librosa.decompose.nn_filter(S_full,
                                       aggregate=np.median,
                                       metric='cosine',
                                       width=int(librosa.time_to_frames(2, sr=sr)))

# The output of the filter shouldn't be greater than the input
# if we assume signals are additive.  Taking the pointwise minimium
# with the input spectrum forces this.
S_filter = np.minimum(S_full, S_filter)


In [7]:
margin_i, margin_v = 2, 10
power = 2

mask_i = librosa.util.softmask(S_filter,
                               margin_i * (S_full - S_filter),
                               power=power)

mask_v = librosa.util.softmask(S_full - S_filter,
                               margin_v * S_filter,
                               power=power)

# Once we have the masks, simply multiply them with the input spectrum
# to separate the components

S_foreground = mask_v * S_full
S_background = mask_i * S_full


In [8]:
new_y = librosa.istft(S_background*phase)
sf.write('file2.wav',new_y,samplerate=sr,subtype='PCM_16')

In [10]:
import shutil
def delete_pycache(root_dir: str, folder_name: str):
    """
    Deletes a folder and its contents from a given root directory.

    Parameters
    ----------
    root_dir: `str`
        The path of the root directory to traverse.
    folder_name: `str`
        The name of the folder to delete.

    Raises
    ------
    OSError:
        If the folder cannot be deleted due to permission or other errors.

    Examples
    --------
    >>> delete_folder("Users/Admin/Documents", "pycache")
        Deleting Users/Admin/Documents_pycache_
        Deleting Users/Admin/Documents/project_pycache_
    """
    # Traverse through the directory tree
    for dirpath, dirnames, _ in os.walk(root_dir):
        # Check if the specified folder exists in the current directory
        if folder_name in dirnames:
            # Get the full path of the pycache folder
            pycache_folder_path = os.path.join(dirpath, folder_name)

            # Print the path of the folder being deleted
            print(f"Deleting {pycache_folder_path}")

            # Delete the pycache folder and its contents
            shutil.rmtree(pycache_folder_path)

In [11]:
delete_pycache("/home/munikumar/Desktop/yt_audio_collector","__pycache__")

Deleting /home/munikumar/Desktop/yt_audio_collector/yt_audio_collector/__pycache__
Deleting /home/munikumar/Desktop/yt_audio_collector/yt_audio_collector/system_2/__pycache__
Deleting /home/munikumar/Desktop/yt_audio_collector/yt_audio_collector/system_1/__pycache__
Deleting /home/munikumar/Desktop/yt_audio_collector/yt_audio_collector/utils/__pycache__
